# Capstone Project - The Battle of the Neighborhoods (Week 1)

# Introduction: Business Problem

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an Coffee Shop in Stockholm, Sweden.
Since there are lots of restaurants in Stockholm we will try to detect locations that are not already crowded with restaurants. We are also particularly interested in areas with no coffee shop in vicinity. We would also prefer locations as close to city center as possible, assuming that first two conditions are met.
We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

# Data

Based on definition of our problem, factors that will influence our decission are:
number of existing coffee shop in the neighborhood (including restaurant offer coffee)
number of and distance to other coffee shop in the neighborhood, if any
distance of neighborhood from city center
We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.
Following data sources will be needed to extract/generate the required information:
centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using Google Maps API reverse geocoding 
number of coffee shop and their type and location in every neighborhood will be obtained using Foursquare API 
coordinate of Dhaka center will be obtained using Google Maps API geocoding 

# Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around Berlin city center.
Let's first find the latitude & longitude of Berlin city center, using specific, well known address and Google Maps geocoding API.

In [1]:
#!pip install folium
import folium
#!pip install geopy
from geopy.geocoders import Nominatim
#!pip install shapely
#import shapely.geometry
#!pip install pyproj
import pyproj
import math
import json # library to handle JSON files
import requests
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#import beautiful soup
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
print('library added')

library added


In [2]:
address = 'Stockholm, Sweden'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
stockholm_center = [latitude, longitude]
print('The geograpical coordinate of ', address ,'are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of  Stockholm, Sweden are 59.3251172, 18.0710935.


Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~6km from Stockholm. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.
To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in meters).

In [3]:
def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Stockholm center longitude={}, latitude={}'.format(stockholm_center[1], stockholm_center[0]))
x, y = lonlat_to_xy(stockholm_center[1], stockholm_center[0])
print('Stockholm center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Stockholm center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Stockholm center longitude=18.0710935, latitude=59.3251172
Stockholm center UTM X=674735.1349179444, Y=6580284.089114029
Stockholm center longitude=18.0710935, latitude=59.3251172


In [4]:
stockholm_center_x, stockholm_center_y = lonlat_to_xy(stockholm_center[1], stockholm_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = stockholm_center_x - 6000
x_step = 600
y_min = stockholm_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(stockholm_center_x, stockholm_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


Let's create a hexagonal grid of cells: we offset every other row, and adjust vertical row spacing so that every cell center is equally distant from all it's neighbors

In [5]:
map_stockholm = folium.Map(location=stockholm_center, zoom_start=13)
folium.Marker(stockholm_center, popup='Stockholm').add_to(map_stockholm)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_berlin) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_stockholm)
    #folium.Marker([lat, lon]).add_to(map_berlin)
map_stockholm

Let's visualize the data we have so far: city center location and candidate neighborhood centers:

OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~6km from Stortorget, Stockholm.
Let's now use Google Maps API to get approximate addresses of those locations.

# Entering in our information into the Foursquare API to access it.¶ 

In [6]:
CLIENT_ID = 'KD1KQGFKVSSP3YYM3OOMLDSKAVCOWQCPYR4IKZVUN3VY2CSS' # your Foursquare ID
CLIENT_SECRET = 'F5ZTS44MOR2OQ1SPELD1RF3ER52O3HO4VDDVVNTGESN0D5J4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KD1KQGFKVSSP3YYM3OOMLDSKAVCOWQCPYR4IKZVUN3VY2CSS
CLIENT_SECRET:F5ZTS44MOR2OQ1SPELD1RF3ER52O3HO4VDDVVNTGESN0D5J4


In [7]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=KD1KQGFKVSSP3YYM3OOMLDSKAVCOWQCPYR4IKZVUN3VY2CSS&client_secret=F5ZTS44MOR2OQ1SPELD1RF3ER52O3HO4VDDVVNTGESN0D5J4&v=20180605&ll=59.3251172,18.0710935&radius=500&limit=100'

# Result from Foursquare

In [8]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d43911b6e4650002ce55552'},
 'response': {'headerLocation': 'Storkyrkan',
  'headerFullLocation': 'Storkyrkan, Stockholm',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 103,
  'suggestedBounds': {'ne': {'lat': 59.3296172045, 'lng': 18.079897699230337},
   'sw': {'lat': 59.3206171955, 'lng': 18.062289300769663}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b06fe89f964a520fef422e3',
       'name': 'Science Fiction Bokhandeln',
       'location': {'address': 'Västerlånggatan 48',
        'crossStreet': 'Gamla Stan',
        'lat': 59.32404679408319,
        'lng': 18.070682162632647,
        'labeledLatLngs': [{'label': 'display',
          'lat': 59.32404679408319,
          'lng': 18.070682162632647}],
        '

# Now we start pulling the data from Foursquare into a dataframe so we can manipulate and use it.

In [9]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
#pull the actual data from the Foursquare API

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
filtered_columns = ['venue.name', 'venue.id', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues

,venue.name,venue.id,venue.categories,venue.location.lat,venue.location.lng
0,Science Fiction Bokhandeln,4b06fe89f964a520fef422e3,Bookstore,59.324047,18.070682
1,Tweed,51434b65e4b00cf8d47795f1,Cocktail Bar,59.324471,18.067696
2,Stortorget,4b0e3607f964a520fa5523e3,Plaza,59.324973,18.070727
3,Barrels Burgers & Beer,549447fb498ed7d33f190dc2,Burger Joint,59.324685,18.068513
4,Stockholms Gästabud,4e8596c602d59e1a9afa5d5e,Gastropub,59.325569,18.073991
5,Tradition,534d0a34498ef782062ac89c,Scandinavian Restaurant,59.325940,18.073710
6,Corner Club,514c0868e4b07a3d6af2fca3,Cocktail Bar,59.323342,18.069431
7,The Burgundy,522784bb11d2164a04286219,Wine Bar,59.324434,18.068161
8,Slingerbulten,4ba0c261f964a520a77b37e3,Scandinavian Restaurant,59.324376,18.068967
9,The Hairy Pig Deli,537f7e60498e54e075c24907,Deli / Bodega,59.325555,18.073958


# Now let's get a list of venue ids so we can pull likes and add to our dataframe.

In [16]:
#fix the column names so they look relatively normal
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,id,categories,lat,lng
0,Science Fiction Bokhandeln,4b06fe89f964a520fef422e3,Bookstore,59.324047,18.070682
1,Tweed,51434b65e4b00cf8d47795f1,Cocktail Bar,59.324471,18.067696
2,Stortorget,4b0e3607f964a520fa5523e3,Plaza,59.324973,18.070727
3,Barrels Burgers & Beer,549447fb498ed7d33f190dc2,Burger Joint,59.324685,18.068513
4,Stockholms Gästabud,4e8596c602d59e1a9afa5d5e,Gastropub,59.325569,18.073991
5,Tradition,534d0a34498ef782062ac89c,Scandinavian Restaurant,59.325940,18.073710
6,Corner Club,514c0868e4b07a3d6af2fca3,Cocktail Bar,59.323342,18.069431
7,The Burgundy,522784bb11d2164a04286219,Wine Bar,59.324434,18.068161
8,Slingerbulten,4ba0c261f964a520a77b37e3,Scandinavian Restaurant,59.324376,18.068967
9,The Hairy Pig Deli,537f7e60498e54e075c24907,Deli / Bodega,59.325555,18.073958


In [17]:
nearby_venues['categories'].unique()

array(['Bookstore', 'Cocktail Bar', 'Plaza', 'Burger Joint', 'Gastropub',
       'Scandinavian Restaurant', 'Wine Bar', 'Deli / Bodega', 'Pub',
       'Jazz Club', 'Bakery', 'Comic Shop', 'Palace', 'Record Shop',
       'Polish Restaurant', 'Mexican Restaurant', 'Café', 'Historic Site',
       'Theme Restaurant', 'Noodle House', 'Bike Shop',
       'Outdoor Sculpture', 'Tea Room', 'History Museum', 'Irish Pub',
       'Hostel', 'Ice Cream Shop', 'Italian Restaurant', 'Coffee Shop',
       'Gift Shop', 'French Restaurant', 'BBQ Joint', 'German Restaurant',
       'Toy / Game Store', 'Bar', 'Restaurant', 'Fish & Chips Shop',
       'Seafood Restaurant', 'Hotel', 'Museum', 'Sushi Restaurant',
       'Church', 'Candy Store', 'Art Museum', 'Opera House',
       'Monument / Landmark', 'Park', 'Breakfast Spot', 'Boat or Ferry',
       'Other Great Outdoors', 'Building'], dtype=object)

and let's now save/persist this data into local file.

In [18]:
# creating a list of categorie to remove from our dataframe because they are not restaurants
# I am sure there is a function that can be written to do this at scale but since it was a small list, I did it manually

removal_list = ['Bookstore', 'Bakery', 'Jazz Club', "Bike Shop", 'Comic Shop', 'Plaza', 'Gastropub',
               'Record Shop', 'Pub', 'Pet Store', 'Historic Site', 'Outdoor Sculpture', 'Tea Room', 'History Museum', 'Irish Pub',
       'Hostel','Gift Shop','BBQ Joint','Toy / Game Store', 'Bar','Fish & Chips Shop','Hotel', 'Museum','Church', 'Candy Store', 'Art Museum', 'Opera House',
       'Monument / Landmark', 'Park', 'Breakfast Spot', 'Boat or Ferry',
       'Other Great Outdoors', 'Building']

nearby_venues2 = nearby_venues.copy()


#getting a clear dataframe of just restaurants
nearby_venues2 = nearby_venues2[~nearby_venues2['categories'].isin(removal_list)]
nearby_venues2

,name,id,categories,lat,lng
1,Tweed,51434b65e4b00cf8d47795f1,Cocktail Bar,59.324471,18.067696
3,Barrels Burgers & Beer,549447fb498ed7d33f190dc2,Burger Joint,59.324685,18.068513
5,Tradition,534d0a34498ef782062ac89c,Scandinavian Restaurant,59.325940,18.073710
6,Corner Club,514c0868e4b07a3d6af2fca3,Cocktail Bar,59.323342,18.069431
7,The Burgundy,522784bb11d2164a04286219,Wine Bar,59.324434,18.068161
8,Slingerbulten,4ba0c261f964a520a77b37e3,Scandinavian Restaurant,59.324376,18.068967
9,The Hairy Pig Deli,537f7e60498e54e075c24907,Deli / Bodega,59.325555,18.073958
10,19 Glas Bar & Matsal,4b9a5e8ff964a52072af35e3,Wine Bar,59.324633,18.068670
14,Pharmarium,51ccb742498e64f625f8ce57,Cocktail Bar,59.324851,18.070891
16,Kungliga Slottet (Kungliga slottet),4adcdaeef964a520c95a21e3,Palace,59.326353,18.071140


In [19]:
#let's get a list of venues

venue_id_list = nearby_venues2['id'].tolist()
venue_id_list

['51434b65e4b00cf8d47795f1',
 '549447fb498ed7d33f190dc2',
 '534d0a34498ef782062ac89c',
 '514c0868e4b07a3d6af2fca3',
 '522784bb11d2164a04286219',
 '4ba0c261f964a520a77b37e3',
 '537f7e60498e54e075c24907',
 '4b9a5e8ff964a52072af35e3',
 '51ccb742498e64f625f8ce57',
 '4adcdaeef964a520c95a21e3',
 '5729f110498e722233305953',
 '4c78322f93ef236a9772ab0f',
 '550867d7498e86f0d8c9a61d',
 '50c0cc79e4b0a80fdc3460df',
 '4ece875cb634dd298f5ea304',
 '4dc28371887770a6be00accd',
 '4b63079df964a520945e2ae3',
 '58c90efcd243323eaf526efb',
 '583875a96d632566864f820c',
 '4d381d443ffba143aed55856',
 '4c63fd4479d1e21eacddd915',
 '4b180045f964a52039cb23e3',
 '4adcdaedf964a520555a21e3',
 '4e198d6e18a8166f7396ffaa',
 '4adcdaf0f964a520655b21e3',
 '4adcdaebf964a520e35921e3',
 '5731e745498ea03b4bfe08cb',
 '51cb1bd3498e9a51850326c1',
 '4d66061b48ee6ea85d166168',
 '4b583319f964a520bf4d28e3',
 '4c9cb5199975a143948034cf',
 '4b9a284ef964a520d7a135e3',
 '55df60fd498e44d120c585cb',
 '51829a91498e80962fe0411e',
 '4b4f5698f964

In [20]:
#set up to pull the likes from the API based on venue ID

url_list = []
like_list = []
json_list = []

for i in venue_id_list:
    venue_url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(i, CLIENT_ID, CLIENT_SECRET, VERSION)
    url_list.append(venue_url)
for link in url_list:
    result = requests.get(link).json()
    likes = result['response']['likes']['count']
    like_list.append(likes)
print(like_list)

[198, 448, 96, 101, 40, 128, 100, 109, 178, 497, 27, 120, 75, 69, 271, 33, 144, 14, 24, 104, 76, 112, 99, 217, 114, 221, 23, 30, 14, 35, 34, 45, 8, 152, 36, 53, 60, 14, 19, 64, 57, 60, 20, 24, 6, 31, 8]


In [21]:
#double check that we did not lose any venues based on if likes were available

print(len(like_list))
print(len(venue_id_list))

47
47


# Data Prep Intro

We thought likes are a proxy for quality. The more likes there are, the better the restaurant is. This might be incorrect but API call issues (how many I can use for free) holds me back from getting price / rating data. I will then bin this data into a quality categorical variables so we can cluster appropriately.
I am also going to create new categorical variables for the restaurants to better group them based on type of cuisine. This way you can look for good mexican food or now what type of food might be best to eat in Hoboken if you are new to the area.
Data Prep Part 1
Now let's start prepping our data for clustering. This will include combining data from different lists, creating new categorical data to be used, binning data and then encoding the data for clustering.

In [22]:
#let's make a copy of our initial dataframe just in case anything goes wrong

stockholm_venues = nearby_venues2.copy()
stockholm_venues.head()

,name,id,categories,lat,lng
1,Tweed,51434b65e4b00cf8d47795f1,Cocktail Bar,59.324471,18.067696
3,Barrels Burgers & Beer,549447fb498ed7d33f190dc2,Burger Joint,59.324685,18.068513
5,Tradition,534d0a34498ef782062ac89c,Scandinavian Restaurant,59.325940,18.073710
6,Corner Club,514c0868e4b07a3d6af2fca3,Cocktail Bar,59.323342,18.069431
7,The Burgundy,522784bb11d2164a04286219,Wine Bar,59.324434,18.068161


# Data prep part 2

In [23]:
# add in the list of likes

stockholm_venues['total likes'] = like_list
stockholm_venues.head()

,name,id,categories,lat,lng,total likes
1,Tweed,51434b65e4b00cf8d47795f1,Cocktail Bar,59.324471,18.067696,198
3,Barrels Burgers & Beer,549447fb498ed7d33f190dc2,Burger Joint,59.324685,18.068513,448
5,Tradition,534d0a34498ef782062ac89c,Scandinavian Restaurant,59.325940,18.073710,96
6,Corner Club,514c0868e4b07a3d6af2fca3,Cocktail Bar,59.323342,18.069431,101
7,The Burgundy,522784bb11d2164a04286219,Wine Bar,59.324434,18.068161,40


# Data prep part 3

In [24]:
# now let's bin total likes

print(stockholm_venues['total likes'].max())
print(stockholm_venues['total likes'].min())
print(stockholm_venues['total likes'].median())
print(stockholm_venues['total likes'].mean())

497
6
60.0
93.7872340425532


In [25]:
import matplotlib.pyplot as plt
stockholm_venues['total likes'].hist(bins=4)
plt.show()

<Figure size 640x480 with 1 Axes>

In [26]:
# what are the bins we want to use?
print(np.percentile(stockholm_venues['total likes'], 25))
print(np.percentile(stockholm_venues['total likes'], 50))
print(np.percentile(stockholm_venues['total likes'], 75))

28.5
60.0
113.0


In [27]:
# now we have our bin values so let's set them to the appropriate values
# less than 24, 24-45, 45-76, 76>
# poor, below avg, abv avg, great

poor = stockholm_venues['total likes']<=24
below_avg = stockholm_venues[(stockholm_venues['total likes']>24) & (stockholm_venues['total likes']<=45)]
abv_avg = stockholm_venues[(stockholm_venues['total likes']>45) & (stockholm_venues['total likes']<=76)]
great = stockholm_venues['total likes']>76

In [28]:
stockholm_venues

,name,id,categories,lat,lng,total likes
1,Tweed,51434b65e4b00cf8d47795f1,Cocktail Bar,59.324471,18.067696,198
3,Barrels Burgers & Beer,549447fb498ed7d33f190dc2,Burger Joint,59.324685,18.068513,448
5,Tradition,534d0a34498ef782062ac89c,Scandinavian Restaurant,59.325940,18.073710,96
6,Corner Club,514c0868e4b07a3d6af2fca3,Cocktail Bar,59.323342,18.069431,101
7,The Burgundy,522784bb11d2164a04286219,Wine Bar,59.324434,18.068161,40
8,Slingerbulten,4ba0c261f964a520a77b37e3,Scandinavian Restaurant,59.324376,18.068967,128
9,The Hairy Pig Deli,537f7e60498e54e075c24907,Deli / Bodega,59.325555,18.073958,100
10,19 Glas Bar & Matsal,4b9a5e8ff964a52072af35e3,Wine Bar,59.324633,18.068670,109
14,Pharmarium,51ccb742498e64f625f8ce57,Cocktail Bar,59.324851,18.070891,178
16,Kungliga Slottet (Kungliga slottet),4adcdaeef964a520c95a21e3,Palace,59.326353,18.071140,497


In [29]:
# let's star the process for re-categorizing the categories

stockholm_venues['categories'].unique()

array(['Cocktail Bar', 'Burger Joint', 'Scandinavian Restaurant',
       'Wine Bar', 'Deli / Bodega', 'Palace', 'Polish Restaurant',
       'Mexican Restaurant', 'Café', 'Theme Restaurant', 'Noodle House',
       'Ice Cream Shop', 'Italian Restaurant', 'Coffee Shop',
       'French Restaurant', 'German Restaurant', 'Restaurant',
       'Seafood Restaurant', 'Sushi Restaurant'], dtype=object)

In [31]:
# let's create our new categories and create a function to apply those to our existing data

bars = ['Pub', 'Cocktail Bar', 'Bar', 'Dive Bar', 'Sports Bar']
other = ['Bagel Shop', 'Tea Room', 'Donut Shop', 'Coffee Shop', 'Bubble Tea Shop', 'Sandwich Place', 'Boutique', 'Ice Cream Shop']
euro_asia_indian_food = ['Falafel Restaurant', 'Korean Restaurant','Sushi Restaurant', 'Indian Restaurant', 'Japanese Restaurant', 'Poke Place', 'Thai Restaurant', 'Vietnamese Restaurant']
mex_southam_food = ['Cuban Restaurant', 'Mexican Restaurant', 'South American Restaurant', 'Latin American Restaurant']
american_food = ['Burger Joint', 'Restaurant', 'American Restaurant']
italian_food = ['Italian Restaurant', 'Seafood Restaurant', 'Pizza Place']

def conditions2(s):
    if s['categories'] in bars:
        return 'bars'
    if s['categories'] in other:
        return 'other'
    if s['categories'] in euro_asia_indian_food:
        return 'euro asia indian food'
    if s['categories'] in mex_southam_food:
        return 'mex southam food'
    if s['categories'] in american_food:
        return 'american food'
    if s['categories'] in italian_food:
        return 'italian food'


In [32]:
stockholm_venues

,name,id,categories,lat,lng,total likes
1,Tweed,51434b65e4b00cf8d47795f1,Cocktail Bar,59.324471,18.067696,198
3,Barrels Burgers & Beer,549447fb498ed7d33f190dc2,Burger Joint,59.324685,18.068513,448
5,Tradition,534d0a34498ef782062ac89c,Scandinavian Restaurant,59.325940,18.073710,96
6,Corner Club,514c0868e4b07a3d6af2fca3,Cocktail Bar,59.323342,18.069431,101
7,The Burgundy,522784bb11d2164a04286219,Wine Bar,59.324434,18.068161,40
8,Slingerbulten,4ba0c261f964a520a77b37e3,Scandinavian Restaurant,59.324376,18.068967,128
9,The Hairy Pig Deli,537f7e60498e54e075c24907,Deli / Bodega,59.325555,18.073958,100
10,19 Glas Bar & Matsal,4b9a5e8ff964a52072af35e3,Wine Bar,59.324633,18.068670,109
14,Pharmarium,51ccb742498e64f625f8ce57,Cocktail Bar,59.324851,18.070891,178
16,Kungliga Slottet (Kungliga slottet),4adcdaeef964a520c95a21e3,Palace,59.326353,18.071140,497


# Results and Discussion 

Our analysis shows that although there is a great number of restaurants in Stockholm, there are pockets of low restaurant density fairly close to city center.Another borough was identified as potentially interesting but our attention was focused on offer a combination of popularity among tourists, closeness to city center, strong socio-economic dynamics and a number of pockets of low restaurant density.
After directing our attention to this more narrow area of interest we first created a dense grid of location candidates (spaced 100m appart); those locations were then filtered so that those with more than two restaurants in radius of 250m and those with a restaurant closer than 400m were removed.
Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates. Addresses of centers of those zones were also generated using reverse geocoding to be used as markers/starting points for more detailed local analysis based on other factors.
Result of all this is all zones containing largest number of potential new restaurant locations based on number of and distance to existing venues - both restaurants in general and Italian restaurants particularly. This, of course, does not imply that those zones are actually optimal locations for a new restaurant! Purpose of this analysis was to only provide info on areas close to Stockholm center but not crowded with existing restaurants - it is entirely possible that there is a very good reason for small number of restaurants in any of those areas, reasons which would make them unsuitable for a new restaurant regardless of lack of competition in the area. Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

# Conclusion 

Purpose of this project was to identify Stockholm areas close to center with low number of restaurants in order to aid stakeholders in narrowing down the search for optimal location for a new restaurant. By calculating restaurant density distribution from Foursquare data we have first identified general boroughs that justify further analysis and then generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.
Final decission on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.